In [1]:
from pyspark.sql import functions as Fun
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql import Row
from pyspark.sql.types import StringType, ArrayType, DataType, json
import seaborn as sea
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# %%bash
# pip install seaborn

In [3]:
df = spark.read.parquet("./1_9_00000000000634607143.parquet")
df.count()

30000

In [ ]:
df.filter()

In [5]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- site_id: string (nullable = true)
 |-- org_id: string (nullable = true)
 |-- firmware_version: string (nullable = true)
 |-- model: string (nullable = true)
 |-- uptime: integer (nullable = true)
 |-- interval: long (nullable = true)
 |-- config_time: long (nullable = true)
 |-- when: long (nullable = true)
 |-- total_client_count: integer (nullable = true)
 |-- active_client_count: integer (nullable = true)
 |-- terminator_remote_addr: string (nullable = true)
 |-- terminator_timestamp: long (nullable = true)
 |-- dnsServers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- svis: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dev: string (nullable = true)
 |    |    |-- vlan: integer (nullable = true)
 |    |    |-- ips: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- inactive_wired_vlans: array (nullable = true)
 |    |-- element: inte

In [13]:
# paths = ['s3a://mist-secor-production/ap-stats-analytics/ap-stats-analytics-production/dt=2019-05-07/hr=21']

# ap_stats_df = spark.read.option("basePath","s3a://mist-secor-production").parquet(*paths)

In [8]:
df.columns

['id',
 'site_id',
 'org_id',
 'firmware_version',
 'model',
 'uptime',
 'interval',
 'config_time',
 'when',
 'total_client_count',
 'active_client_count',
 'terminator_remote_addr',
 'terminator_timestamp',
 'dnsServers',
 'svis',
 'inactive_wired_vlans',
 'IPv4Routers',
 'dhcpclients',
 'l2tptunnels',
 'lldp_nd',
 'lldp_pwr',
 'radios',
 'wlans',
 'pwr_src',
 'pwr_budget',
 'pwr_needed',
 'pwr_available',
 'memory_total_kb',
 'memory_used_kb',
 'cpu_total_time',
 'cpu_user',
 'cpu_system',
 'fs_available_kb',
 'fs_total_kb',
 'load_total',
 'slab_active_objs',
 'slab_num_objs',
 'slab_obj_size',
 'cloud_last_rtt',
 'cloud_pmtu',
 'temperature_attitude',
 'temperature_cpu',
 'temperature_ambient',
 'orientation_accel_x',
 'orientation_accel_y',
 'orientation_accel_z',
 'orientation_magne_x',
 'orientation_magne_y',
 'orientation_magne_z',
 'switchports',
 'delta',
 'deltaErrorMsg',
 'ap_id',
 'delta_interval']

In [11]:
# sorted(ap_stats_df.schema.names)

In [36]:
df1 = df.select(['site_id', 'id', 'when', 'switchports']) #.filter(lambda x: x['switchports'][0].get("link") )
df2 =df1.select('site_id')


SyntaxError: invalid syntax (<ipython-input-36-68e4d944b733>, line 1)

In [33]:
df1.show(5)

+--------------------+-----------------+----------------+--------------------+
|             site_id|               id|            when|         switchports|
+--------------------+-----------------+----------------+--------------------+
|d635c2b9-3bfb-483...|5c-5b-35-3e-43-a8|1557343429748085|[[eth0, true, tru...|
|8103dd7a-8242-498...|5c-5b-35-3e-7d-eb|1557343429601174|[[eth0, true, tru...|
|b80482ff-5841-49a...|5c-5b-35-3f-69-68|1557343430071011|[[eth0, true, tru...|
|128fae2e-61dc-4ee...|5c-5b-35-2e-26-e8|1557343429897322|[[eth0, true, tru...|
|3dd377f2-4abc-40e...|5c-5b-35-0e-c3-a7|1557343429705259|[[eth0, true, tru...|
+--------------------+-----------------+----------------+--------------------+
only showing top 5 rows



In [37]:
df1_rdd = df1.rdd
df1_rdd = df1_rdd.map(lambda x: [x.id,x.site_id, x.when, x.switchports[0].tx_bytes,
                                 x.switchports[0].tx_peakbps, x.switchports[0].rx_errors ])
df1_rdd.take(1)

[[u'5c-5b-35-3e-43-a8',
  u'd635c2b9-3bfb-4833-a3bd-cb7f3fdb76d9',
  1557343429748085,
  29226,
  0,
  0]]

In [44]:
x, y = [], []
dat = df1_rdd.take(100)
# for dat in df1_rdd.collect():

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 33.0 failed 1 times, most recent failure: Lost task 1.0 in stage 33.0 (TID 100, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/mistsys/spark_jobs/venv/spark-2.4.0-bin-without-hadoop/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/opt/mistsys/spark_jobs/venv/spark-2.4.0-bin-without-hadoop/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/mistsys/spark_jobs/venv/spark-2.4.0-bin-without-hadoop/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/mistsys/spark_jobs/venv/spark-2.4.0-bin-without-hadoop/python/pyspark/rdd.py", line 1354

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 30.0 failed 1 times, most recent failure: Lost task 2.0 in stage 30.0 (TID 88, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/mistsys/spark_jobs/venv/spark-2.4.0-bin-without-hadoop/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/opt/mistsys/spark_jobs/venv/spark-2.4.0-bin-without-hadoop/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/mistsys/spark_jobs/venv/spark-2.4.0-bin-without-hadoop/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/mistsys/spark_jobs/venv/spark-2.4.0-bin-without-hadoop/python/lib/pyspark.zip/py

In [ ]:
def stats(arr):
     
    vals= arr[0]
    
    data_point = Row(
                     tx_peakbps = vals['tx_peakbps'],
                     rx_peakbps = vals['rx_peakbps'],
                     rx_bytes = vals['rx_bytes'],
                     tx_bytes = vals['tx_bytes'],
                     rx_errors = vals['rx_errors']
                    )
    return data_point

site_ap_stats = df1_rdd.map(lambda x: (x[0], stats(x[1])))

In [17]:
df1.take(1)

[Row(site_id=u'd635c2b9-3bfb-4833-a3bd-cb7f3fdb76d9', id=u'5c-5b-35-3e-43-a8', when=1557343429748085, switchports=[Row(name=u'eth0', link=True, full_duplex=True, mbps=1000, rx_bytes=731411, rx_pkts=3648, tx_bytes=29226, tx_pkts=79, rx_errors=0, rx_pause_pkts=0, rx_undersize_errors=0, rx_oversize_errors=0, rx_jabber_errors=0, rx_alignment_errors=0, rx_fcs_errors=0, rx_fragment_errors=0, rx_symbol_errors=0, rx_overrun_errors=207, rx_discards=100399, rx_peakbps=10896, tx_peakbps=0), Row(name=u'eth1', link=False, full_duplex=False, mbps=0, rx_bytes=0, rx_pkts=0, tx_bytes=0, tx_pkts=0, rx_errors=0, rx_pause_pkts=0, rx_undersize_errors=0, rx_oversize_errors=0, rx_jabber_errors=0, rx_alignment_errors=0, rx_fcs_errors=0, rx_fragment_errors=0, rx_symbol_errors=0, rx_overrun_errors=0, rx_discards=0, rx_peakbps=0, tx_peakbps=0), Row(name=u'module', link=True, full_duplex=True, mbps=1000, rx_bytes=9643, rx_pkts=41, tx_bytes=710886, tx_pkts=3403, rx_errors=0, rx_pause_pkts=0, rx_undersize_errors=0,

### Interested fields (for now)

site_id, id, svis, radios, active_client_count, total_client_count, cloud_last_rtt

In [ ]:
# distinct_sites = ap_stats_df.select('site_id').distinct().count()

In [ ]:
# number of sites
# distinct_sites

In [ ]:
# ap_stats_df.take(1)

### Identifying the paterns in device state

In [ ]:
device_state = ap_stats_df.select(['site_id', 'id','when', 'fs_available_kb','fs_total_kb','cpu_system',
                    'cpu_total_time','cpu_user','temperature_cpu',
                    'temperature_attitude','temperature_ambient']).groupby(['site_id', 'id'])
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf("double", PandasUDFType.GROUPED_AGG)
def myUDF(v):
    return v.mean()
    
device_state.agg(myUDF(ap_stats_df['cpu_system'])).show()

In [ ]:
def radio_stats(arr):
    vals = {}
    vals['bandwidth'] = np.mean(list(map(lambda x: x['bandwidth'], arr)))
    vals['noise_floor'] = np.mean(list(map(lambda x: x['noise_floor'], arr)))
    vals['tx_phy_err'] = np.mean(list(map(lambda x: x['tx_phy_err'], arr)))
    vals['utilization_all'] = np.mean(list(map(lambda x: x['utilization_all'], arr)))

    return vals

In [ ]:
def get_max(arr, key):
    max_b = float(np.max(list(map(lambda x: x[key], arr) )))   
    return max_b

In [ ]:
def stats(arr):
    
    vals = {}
#     for item in ['temperature_cpu', 'load_total', 'total_client_count', 'cpu_system', 'cpu_total_time','radios']:
    for item in ['radios']:
 
#         if item == 'radios':
        a = list(map(lambda x: radio_stats(x[item]),arr))

        vals['radios_bandwidth'] = get_max(a, 'bandwidth')
        vals['radios_noise_floor'] = get_max(a, 'noise_floor')
        vals['radios_tx_phy_err'] = get_max(a, 'tx_phy_err')
        vals['radios_utilization_all'] = get_max(a, 'utilization_all')

#         else:    
#             a = list(map(lambda x: x[item],arr))
#             mean_a = float(np.mean(a))
#             median_a = float(np.median(a))
#             max_a = float(np.max(a))
#             min_a = float(np.min(a))

#         vals[item] =  max_a
    
    data_point = Row(
#                      temperature_cpu= vals['temperature_cpu'], 
#                      load_total= vals['load_total'],
#                      total_client_count= vals['total_client_count'],
#                      cpu_system= vals['cpu_system'],
#                      cpu_total_time = vals['cpu_total_time'],
                     radios_bandwidth = vals['radios_bandwidth'],
                     radios_noise_floor = vals['radios_noise_floor'],
                     radios_tx_phy_err = vals['radios_tx_phy_err'],
                     radios_utilization_all = vals['radios_utilization_all']
                    )
    return data_point

In [ ]:
import time
def to_time(t):
    '''
    t is in microseconds
    '''
    return time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime(t/1000000))

### Group data based on site_id, ap_mac. Drop all groups that have less than 10 elements

In [ ]:
device_state = ap_stats_df.select(['site_id', 'id','when','radios'])
site_ap_group = device_state.rdd.filter(lambda x: x is not None).map(lambda x: ((x['site_id'], x['id']), x)).groupByKey()
site_ap_group = site_ap_group.filter(lambda x: x[1] is not None and len(x[1]) > 10)


### Get all distinct ap_macs

In [ ]:
# site_ap_group_aps = site_ap_group.map(lambda x: x[0][1]).distinct().collect()


In [ ]:
# site_ap_group_aps[1:10]

In [ ]:
site_ap = site_ap_group.filter(lambda x: x[0][1] == '5c-5b-35-2e-66-df')
site_ap_stats = site_ap.map(lambda x: (x[0], stats(x[1])))

In [ ]:
site_ap_stats_df = site_ap_stats.map(lambda x: (x[0][0], x[0][1],x[0][2],x[0][3],
                                                x[1]['radios_bandwidth'],
                                                x[1]['radios_noise_floor'],
                                                x[1]['radios_tx_phy_err'],
                                                x[1]['radios_utilization_all']
                                               )
                                    )

In [ ]:
list(site_ap.take(1)[1])

In [ ]:
%spark -o site_ap_stats_df

In [ ]:
%local
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

site_ap_stats_df_sorted =site_ap_stats_df.sort_values(by=['_3'])

sea.distplot(site_ap_stats_df_sorted['_7'], hist = True, kde = True,kde_kws = {'linewidth': 3}, label="HHHH")
# site_ap_stats_df_sorted

In [ ]:
error_ap = ap_stats_df.where(ap_stats_df.id == '00-78-88-e6-36-b7').select('radios')


In [ ]:
_error_ap = error_ap.rdd.map(lambda x: x['radios'])


In [ ]:
site_ap.filter(lambda x: x[0][1] == '00-78-88-e6-36-b7').take(1)

## Helpers for use in the reduce/lambda functions

In [ ]:
def get_avg(items, keys):
    features = []
    for k in keys:
        features.append(Row(mean=float(np.mean(list(map(lambda x: x[k], items)))),key=k))
    return features

In [ ]:
_error_ap = _error_ap.map(lambda x: get_avg(x, ['utilization_non_wifi','chanutil_intf'])).toDF()


In [ ]:
%local
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
sea.distplot(_pune_avg_util_non_wifi, hist = True, kde = True,kde_kws = {'linewidth': 3}, label="HHHH")

In [ ]:

rdd =spark.WholeTextFiles("s3n://ap-logs-staging/ap/*/2018/12/03/")

In [ ]:
#display(rdd)